In [25]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
#import tensorflow_hub as hub
from google.protobuf import text_format
from tensorflow.keras import mixed_precision
import tensorflow_model_optimization as tfmot
import numpy as np
import os
import time

gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [26]:
IMAGE_SHAPE=(224, 224)
BATCH_SIZE=32
NUM_CLASSES = 3

In [27]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255, 
    width_shift_range=.01,
    height_shift_range=.01,
    zoom_range=0.01,
    channel_shift_range=0.01,
    validation_split=0.025)
train_data = image_generator.flow_from_directory(
    "/home/gragundier/Data/ffxiv_fisher4m",
    target_size=IMAGE_SHAPE,
    subset="training",
    class_mode="categorical"
    )
validation_data = image_generator.flow_from_directory(
    "/home/gragundier/Data/ffxiv_fisher4m", 
    target_size=IMAGE_SHAPE,
    subset="validation",
    class_mode="categorical"
    )

Found 1549 images belonging to 3 classes.
Found 39 images belonging to 3 classes.


In [28]:
model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(224, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.SpatialDropout2D(0.2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.SpatialDropout2D(0.2),
    # The third convolution
    #tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.SpatialDropout2D(0.2),
    # The final convolution (same to make compute of reshape easier)
    tf.keras.layers.Conv2D(24, (3,3), padding='same', activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    
    # Reshape to fit into LSTM/GRU
    #tf.keras.layers.Reshape((26*26,8)),
    #tf.keras.layers.LSTM(224*4, activation='tanh', use_bias = True, recurrent_dropout=0),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 222, 222, 224)     6272      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 111, 111, 224)     0         
_________________________________________________________________
spatial_dropout2d_28 (Spatia (None, 111, 111, 224)     0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 109, 109, 64)      129088    
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
spatial_dropout2d_29 (Spatia (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 54, 54, 24)      

In [29]:
'''
copied_metrics = [
    tfma.metrics.ExampleCount(name='example_count'),
    tfma.metrics.WeightedExampleCount(name='weighted_example_count'),
    tf.keras.metrics.SparseCategoricalCrossentropy(
        name='sparse_categorical_crossentropy'),
    tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision', top_k=1),
    tf.keras.metrics.Precision(name='precision', top_k=3),
    tf.keras.metrics.Recall(name='recall', top_k=1),
    tf.keras.metrics.Recall(name='recall', top_k=3),
    tfma.metrics.MultiClassConfusionMatrixPlot(
        name='multi_class_confusion_matrix_plot'),
]
'''
#https://www.tensorflow.org/tfx/model_analysis/metrics
#metrics_specs = tfma.metrics.specs_from_metrics(metrics)
'''
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, 0),
    'block_size': (1, 1),
    'block_pooling_type': 'AVG'
}

tfmot.sparsity.keras.prune_low_magnitude(
    model, **pruning_params
)
'''

model.compile(loss = 'categorical_crossentropy', optimizer='sgd', metrics=['accuracy', 'FalsePositives'])

In [33]:
model_name = "sequential_16"
t = time.time()
log_dir = "/home/gragundier/Data/logs/{}".format("{}-{}".format(model_name, t))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_filepath = '/tmp/checkpoint{}'.format(t)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    #save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


history = model.fit(train_data, epochs=10, validation_data = validation_data, callbacks=[tensorboard_callback, model_checkpoint_callback], verbose = 1)
#history = model.fit(train_data, epochs=500, validation_data = validation_data, verbose = 1)

Epoch 1/10
49/49 [==============================] - 16s 324ms/step - loss: 0.0128 - accuracy: 0.9994 - false_positives: 1.0000 - val_loss: 0.5981 - val_accuracy: 0.7692 - val_false_positives: 8.0000
INFO:tensorflow:Assets written to: /tmp/checkpoint1611759096.3163717/assets
Epoch 2/10
49/49 [==============================] - 16s 314ms/step - loss: 0.0212 - accuracy: 0.9923 - false_positives: 11.0000 - val_loss: 0.2430 - val_accuracy: 0.9231 - val_false_positives: 3.0000
INFO:tensorflow:Assets written to: /tmp/checkpoint1611759096.3163717/assets
Epoch 3/10
49/49 [==============================] - 16s 317ms/step - loss: 0.0245 - accuracy: 0.9903 - false_positives: 15.0000 - val_loss: 0.4375 - val_accuracy: 0.8205 - val_false_positives: 7.0000
Epoch 4/10
49/49 [==============================] - 16s 318ms/step - loss: 0.4409 - accuracy: 0.9270 - false_positives: 98.0000 - val_loss: 0.3954 - val_accuracy: 0.7692 - val_false_positives: 7.0000
Epoch 5/10
49/49 [==============================]

In [ ]:
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path, save_format='tf')

export_path